# Predicting Insurance Costs With Linear Regression

## 1. Introduction

This project aims to forecast insurance costs by using Linear Regression on the [Medical Cost Dataset](https://www.kaggle.com/datasets/mirichoi0218/insurance?resource=download) from Kaggle, which contains information on individual medical insurance bills, where each bill is associated with some demographic and personal characteristics of its recipient.

Of interest is how the different characteristics relate to the **total medical cost**, which is a continuous, positive number. The goal is to construct the best possible predictive model for the cost, given some information about the patient. Predicting medical costs is an important task as it allows hospitals to predict revenue and plan necessary procedures needed by its patient population.

In [1]:
# Include data dictionary?

In [2]:
# Load the relevant libraries for the project
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

%matplotlib inline

In [3]:
# Read the data into a dataframe
insurance = pd.read_csv('insurance.csv')

In [5]:
# Let's take a look at the first 5 rows of the dataframe
insurance.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


## 2. Exploring the Dataset